<div style='width:20em'><img src='img/logo-igm.png'></div>
<div style='font-size:larger'><strong>Algorithmique et programmation 2</strong><br>
L1 Mathématiques - L1 Informatique<br>
Semestre 2
</div>

# Complexité algorithmique

## Motivation

-   On n’exige pas seulement d’un algorithme qu’il résolve un problème,
    c’est-à-dire :

    -   qu’il donne la bonne réponse (**correction**)
    -   en un temps fini (**terminaison**)
    -   sur chaque instance

-   On veut également qu’il soit **efficace**, c’est-à-dire :

    -   rapide (en termes de temps d’exécution)
    -   peu gourmand en ressources (mémoire utilisée)
    
    *(on s’intéressera pour commencer au temps d’exécution)*

## Mesure de performances

On peut utiliser la fonction `time()` du module `time` pour chronométrer l'exécution, pourquoi pas pour tracer des graphiques

In [ ]:
# pour chronométrer
from time import time

# enregistrer le temps de départ
debut = time()

# faire quelque chose
for i in range(10**4):
    pass

# calculer le temps écoulé
print(time() - debut)

Un autre outil un peu plus élaboré est le module `timeit` :

In [ ]:
from timeit import timeit

timeit("""
for i in range(10**4):
    pass
""", number=100)

Sous Jupyter, on peut aussi utiliser les "formules magiques" `%%timeit` (tout une cellule) ou `%timeit` (une seule ligne) :

In [ ]:
%%timeit
for i in range(10**6): 
    pass

In [ ]:
%timeit for i in range(10**6): pass

### Exemple 1 : recherche dans une liste

#### Algorithme "idéal"

- Soit $L$ une liste, $x$ un élément à chercher
- Pour chaque élément tour à tour dans la liste :
    - Si cet élément est égal à $x$, répondre "vrai" et arrêter
    - Sinon continuer
- Si on atteint la fin de la liste sans avoir trouvé $x$, répondre "faux"

Quels sont les cas où l'algorithme prend le plus de temps (pires cas) ? 

Comment varie la "quantité de travail" à fournir en fonction de $L$ et $x$ dans ces cas ?

> *Moralement*, cela devrait être (au pire) proportionnel au nombre d'éléments dans la liste : il suffit de regarder chaque élément de la liste une fois.

#### Vérification expérimentale : fonction itérative

In [ ]:
def recherche_iter(L, x):
    for elem in L:
        if elem == x:
            return True
    return False

In [ ]:
liste_mille = list(range(1000))
liste_deux_mille = list(range(2000))
liste_quatre_mille = list(range(4000))

%timeit recherche_iter(liste_mille, 'pas-là')
%timeit recherche_iter(liste_deux_mille, 'absent')
%timeit recherche_iter(liste_quatre_mille, 'non')

Ici, on voit que si la taille de la liste double, le temps de calcul est multiplié à peu près par deux. Cela semble confirmer l'hypothèse précédente.

#### Vérification expérimentale : fonction récursive

In [ ]:
# pour avoir une pile plus grande
from sys import setrecursionlimit  
setrecursionlimit(3000)

In [ ]:
def recherche_rec(lst, e):
    if len(lst) == 0:
        return False
    elif lst[0] == e:
        return True
    else:
        return recherche_rec(lst[1:], e)

In [ ]:
liste_mille = list(range(1000))
liste_deux_mille = list(range(2000))

%timeit recherche_rec(liste_mille, 'nope')
%timeit recherche_rec(liste_deux_mille, 'cherche ailleurs')

- Cette fois, pour une liste **deux** fois plus grande, le temps d'exécution est multiplié environ par **quatre** : ça n'a pas l'air proportionnel...

- On remarque aussi que cette fonction est globalement **beaucoup** plus lente que la version itérative (c'est assez normal, on y reviendra).

#### Vérification expérimentale : graphique

Pour y voir plus clair, essayons de tracer un graphique :

In [ ]:
# pour dessiner des graphiques
import matplotlib.pyplot as plt
%matplotlib inline
# %config InlineBackend.figure_format='retina'

In [ ]:
def teste_recherche(fonction, tailles):
    """
    Fonction cherchant 1 dans des listes de 0
    de longueur données dans tailles.
    Renvoie une liste des temps d'exécution.
    """
    temps = []
    for n in tailles:
        lst = [0] * n
        t = timeit(lambda: fonction(lst, 1), 
                   number=200, globals=globals())
        temps.append(t)
    return temps

In [ ]:
# On va tester avec les tailles ci-dessous
tailles = list(range(0, 3000, 300))

In [ ]:
# On lance les tests
temps = teste_recherche(recherche_iter, tailles)
plt.plot(tailles, temps, 's-', label='itérative')

temps = teste_recherche(recherche_rec, tailles)
plt.plot(tailles, temps, 's-', label='récursive')

plt.legend()
pass

Le problème se confirme... la courbe pour la fonction `recherche_rec` ne ressemble pas à une droite.

La raison (déjà donné au chapitre précédent) est que nous utilisons une instruction coûteuse sans en tenir compte :

```python
def recherche_rec(lst, e):
    if len(lst) == 0:
        return False
    elif lst[0] == e:
        return True
    else:
        return recherche_rec(lst[1:], e)
        # lst[1:]  prend un temps proportionnel à len(lst) !!!
```

Pour remédier au problème, on peut utiliser un argument supplémentaire (indice) et ne pas modifier la liste :

In [ ]:
def recherche_rec_aux(lst, e, i):
    if len(lst) <= i:
        return False
    elif lst[i] == e:
        return True
    else:
        return recherche_rec_aux(lst, e, i+1)

def recherche_rec2(lst, e):
    return recherche_rec_aux(lst, e, 0)

In [ ]:
recherche_rec2([1, 3, -5], 4)

In [ ]:
recherche_rec2([1, 3, -5], -5)

On vérifie que le problème semble résolu :

In [ ]:
# On lance les tests
temps = teste_recherche(recherche_iter, tailles)
plt.plot(tailles, temps, 's-', label='itérative')

#temps = teste_recherche(recherche_rec, tailles)
#plt.plot(tailles, temps, 's-', label='récursive avec tranche')

temps = teste_recherche(recherche_rec2, tailles)
plt.plot(tailles, temps, 's-', label='récursive avec indice')

plt.legend()
pass

La fonction récursive reste plus lente (c'est normal) mais au moins le temps d'exécution semble vaguement proportionnel à la taille de la liste.

### Exemple 2 : recherche de doublons

In [ ]:
def contient_doublon(lst):
    for i in range(len(lst)):
        for j in range(i+1, len(lst)):
            if lst[i] == lst[j]:
                return True
    return False

In [ ]:
contient_doublon([1, 0, 3, 4, 1, 3])

In [ ]:
contient_doublon([1, 0, 3, 4, 2, -3])

In [ ]:
tailles = list(range(0, 10000, 1000))

temps = []
for n in tailles:
    lst = list(range(n))
    t = timeit('contient_doublon(lst)', 
               globals=globals(), number=10)
    temps.append(t)

plt.plot(tailles, temps, 's-')
pass

Ça n'a toujours pas l'air linéaire... Essayons le "test de doublement" :

In [ ]:
liste_mille = list(range(1000))
liste_deux_mille = list(range(2000))

%timeit contient_doublon(liste_mille)
%timeit contient_doublon(liste_deux_mille)

Pour une liste de taille **double**, le temps de calcul a **beaucoup plus que doublé**. On verra plus précisément dans la suite comment évolue le temps de calcul de cette fonction par rapport à `len(lst)`.

## La complexité d'un algorithme

-   Inconvénients du chronométrage de programmes :

    -   nécessite d’implémenter les algorithmes
    -   mesures dépendant de nombreux facteurs (machine, langage, charge...)
    -   peut être dur à implémenter
    -   observations difficiles à transposer, à généraliser...

-   On veut pouvoir évaluer la qualité **théorique** des algorithmes

    -   quel algorithme choisir **avant de commencer à programmer** ?
    -   pourquoi tel programme est-il aussi lent ?
    -   où agir pour améliorer un algorithme ?

-   La **théorie de la complexité** (algorithmique) permet aussi :

    -   de classer les *algorithmes* selon leur efficacité
    -   de comparer plusieurs algorithmes résolvant un même problème
        *sans les implémenter*
    -   de classer les *problèmes* selon leur difficulté


-   **Idée de base :** *compter* le nombre d’opérations élémentaires effectuées

### Comptage des opérations

-   Chaque **opération élémentaire** compte pour une “unité de temps”
    (pour simplifier)

    -   exemples : `=`, `==`, `<=`, `+`, `-`, `*`, `/`, `lst[i]`...

    -   considérées comme prenant un temps “constant” (indépendant de
        l’entrée)

    -   doivent être bien choisies

-   Chaque **itération** d’une boucle rajoute le nombre d’unités de
    temps prises par le corps de cette boucle

-   Chaque **appel de fonction** rajoute le nombre d’unités de temps
    prises par son exécution

### Exemple: calcul de la factorielle

Version itérative :

```python
def fact_iter(n):  
    fact = 1             # affectation :  1           
    i = 2                # affectation :  1
    while i <= n:        # comparaison :  1
        fact = fact * i  # mult. + aff. : 2
        i = i + 1        # add. + aff. :  2
    return fact          # renvoi valeur: 1
```

-   Si $n > 1$ on a exactement $n - 1$ tours de boucle

-   Nombre total d’opérations pour un appel à `fact(n)` :
    
    $$T(n) = 1+1+(n-1)*5+1+1=5n-1$$

Version récursive :

```python
def fact_rec(n):
    if n <= 1:    # comparaison :   1
        return 1  # renvoi valeur : 1
    else:
        return n * fact_rec(n - 1) 
        # appel récursif : T(n-1)
        # soustr. + mult. + retour : 3
```

Nombre total d’opérations (pour $n > 0$): 

$$
\begin{cases}
  T(1) = 2,\\
  T(n) = 4 + T(n-1)
\end{cases}
$$

Total : $T(n) = 4n-2$

### Remarques

-   Ce genre de calcul est en général inexact

    -   on ne sait pas toujours combien de fois exactement on va
        effectuer une boucle (encadrement)

    -   lors d’un branchement conditionnel, le nombre d’opérations
        effectuées n’est pas toujours le même (encadrement)

    -   l’hypothèse selon laquelle chaque opération élémentaire prend le
        même temps est en général fausse

    -   un appel de fonction engendre un coût supplémentaire (gestion de
        la pile)

-   Les calculs mathématiques requis peuvent être difficiles

-   Heureusement, on dispose d’outils plus simples et plus adaptés

## Complexité asymptotique

On appelle **"complexité"** d’un algorithme une mesure de sa performance
**asymptoptique** dans le **pire cas**

-   Que signifie **asymptotique**?
    -   on considère le nombre d'opérations quand la taille de l'entrée tend vers l'infini
    -   pourquoi ? parce que cela permet de se concentrer sur le terme dominant dans l'expression du nombre d'opérations (vitesse de croissance la plus grande)

-   Que signifie **dans le pire cas** ?
    -   pour une taille donnée, on considère la pire entrée possible de cette taille (celle qui demandera le plus de calculs)
    -   pourquoi? parce que cela fournit une garantie sur le pire temps d'exécution possible pour une entrée de taille donnée

On peut aussi regarder la complexité **au mieux** (pas toujours intéressant), ou
**en moyenne** (intéressant mais plus difficile à calculer)

### Intuition: comportement asymptotique

Soient :

1.  un problème à résoudre sur des données de taille $n$
2.  deux algorithmes le résolvant en temps $f_1(n)$ et $f_2(n)$

<img src='img/plot.png' width='50%'>

Quel algorithme préférez-vous?

<img src='img/plot2.png' width='50%'>

### La notation $O(\cdot)$

-   Calculs exacts du nombre d’opérations parfois longs et pénibles

-   Degré de précision mal maîtrisé (détails dépendant de la machine, du
    langage, de l’implémentation...) et souvent inutile

-   On recourt à une **approximation** représentée par la notation
    $O(\cdot)$

-   Soit $n$ la taille des données à traiter; on dit qu’une fonction
    $f(n)$ est **en $O(g(n))$** (« grand Ô de $g(n)$ ») ou **dominée asymptotiquement par $g(n)$** si

    $$
    \exists\ n_0 \in \mathbb{N},
      \exists\ c \in \mathbb{R},
      \forall\ n\geq n_0: \quad
      \left| f(n) \right| \leq c \left| g(n) \right|
    $$


-   Autrement dit: à
    partir d'un certain rang $f(n)$ est toujours inférieure à $g(n)$, à
    une constante multiplicative fixée près
-   Autrement dit: $f(n)$ ne croît pas plus vite que $g(n)$ fois une constante quand $n$ devient grand

### Illustration

Quelques cas où $f(n)$ (en bleu) $\in O(g(n))$ (en rouge) :

<img src='img/plot3-0.png' width='40%'>

<img src='img/plot3-1.png' width='40%'>

<img src='img/plot3-2.png' width='40%'>

Dans ce dernier cas ci-dessous, on a aussi $g(n) \in O(f(n))$ ! (pourquoi ?)

### Propriétés

Soient $(f_n)_{n \in  \mathbb{N}}$, $(f'_n)_{n \in  \mathbb{N}}$, $(g_n)_{n \in  \mathbb{N}}$ et $(g'_n)_{n \in  \mathbb{N}}$ des suites réelles strictement positives à partir d'un certain rang. 

On a les règles de calcul suivantes:

1. $f_n ∈ O(f_n)$

**exemple :** $n^2 \in O(n^2)$

2. Si $f_n \in O(g_n)$ alors $c \times f_n \in O(g_n)$, pour $c \in \mathbb{R}$

**exemples :** 
- $3n^2 \in O(n^2)$
- $n^2/5 \in O(n^2)$
- $10^6 \in O(1)$

3. Si $f_n \in O(g_n)$ et $f'_n \in O(g'_n)$ alors $f_n + f'_n \in O(\max(g_n, g'_n))$

Cas particuliers :
  - si $f_n \in O(g_n)$ et $f'_n \in O(g_n)$ alors $f_n + f'_n \in O(g_n)$
  - $f_n + f_n \in O(f_n)$
  
**exemples :** 
- $n + n \in O(n)$
- $n^2 + 100 \in O(\max(n^2, 100))$,  
  mais comme $100 \in O(n^2)$ on a aussi $n^2 + 100 \in O(n^2)$

4. Si $f_n \in O(g_n)$ et $f'_n \in O(g'_n)$ alors $f_n \times f'_n \in O(g_n \times g'_n)$

  **exemples :** $(4n) \times (\log n/2) \in O(n \log n)$  
  mais aussi $(4n) \times (\log n/2) \in O(n^2)$

### Échelle de fonctions

On dit que $f_n$ croît plus lentement que $g_n$ ou est *dominée* par $g_n$, noté $f_n \prec g_n$, si $\lim_{n \to \infty} \frac{f_n}{g_n} = 0$

En d'autres termes, quand $n$ devient grand, $f_n$ devient négligeable devant $g_n$

On écrit aussi : $f(n) \in o(g(n))$

Pour tous réels $\varepsilon$, $\delta$, $c$ et $d$ avec $0 < \varepsilon < \delta < 1 < c < d$, on a :

$$
1 \prec \log n \prec (\log n)^c \prec (\log n)^d \prec n^\varepsilon \prec n^\delta \prec n \prec n^c \prec n^d \prec c^n \prec d^n \prec n! \prec n^n \prec c^{c^n}
$$

En particulier :

$$
1 \prec \log n  \prec \sqrt{n} \prec n \prec n \log n \prec n^2 \prec n^3 \prec 2^n \prec n!
$$

#### Propriété importante :

5. Si $f_n \prec g_n$ alors $f_n \in O(g_n)$  
*Si une fonction croît plus lentement qu'une autre, alors elle est dominée asymptotiquement par cette fonction*

**exemples :**
- $n \in O(n^2)$, $n^2 \in O(n^3)$, $n^3 \in O(n^{100})\ldots$
- $\sqrt{n} \in O(n)$
- $n \log n \in O(n^2)$

### Règle pratique

En combinant les règles précédentes on obtient : 

- $O(c \cdot f_n) = O(f_n)$ si $c$ ne dépend pas de $n$
- $O(f_n + g_n) = O(g_n)$ si $f_n \in O(g_n)$

Cela signifie que dans une expression $O$ on peut
- *ignorer les constantes multiplicatives*
- *ignorer les termes dominés*

#### Exemple

Soit un algorithme effectuant $g(n)=4n^3+5n^2\log(n)+2000n+3$ opérations sur des entrées de taille $n$, on a : 

$$
\begin{aligned}
    g(n) & \in O(4 n^3 + 5 n^2 \log(n) + 2000 n + 3)\\
         & \in O(4n^3) \\
         & \in O(n^3)
\end{aligned}
$$

### Observations

Attention, la notation $O(...)$ n'est pas toujours intuitive ! Il faut bien se souvenir qu'il s'agit d'une *majoration* (à la limite et à un facteur près). En particulier, ce n'est pas une notation « symétrique ».

On peut par exmple faire les observations suivantes :

-   $f(n) \in O(g(n))$ n’implique pas $g(n) \in O(f(n))$:

    -   contre-exemple: $n \in O(n^2)$, mais $n^2 \not\in O(n)$;

-   $f(n) \not\in O(g(n))$ n’implique pas $g(n) \not\in O(f(n))$:

    -   contre-exemple: $n^3 \not\in O(n^2)$, mais $n^2 \in  O(n^3)$;

### Justification des conventions : pourquoi utiliser $O$ ?

-   On raisonne sur des algorithmes, pas sur leurs **implémentations**

    -   les processeurs actuels effectuent plusieurs milliards
        d’opérations par seconde

    -   qu’une affectation requière 2 ou 4 unités de temps ne change pas
        grand-chose

    -   et d’ailleurs on ne peut pas toujours le savoir

-   On s’intéresse plutôt à une **vitesse de croissance**

    -   quand $n$ grandit, certains termes croissent plus vite, les autres deviennent négligeables
    -   cela peut suffire à faire des prédictions sur le temps de calcul d'un programme

### Analyse de complexité en pratique

Les instructions de base sont supposées prendre un temps constant, noté $O(1)$. Pour les autres structures de contrôle, on applique les règles vues précédemment.

#### Séquences d'instructions

-   On additionne les complexités des opérations (ou on prend le maximum) :

    ```python
    une_instruction      # complexité : O(f(n))
    instuction_suivante  # complexité : O(g(n))
    ```
    
    Coût total en $O(f(n) + g(n)) = O(\max(f(n), g(n)))$

#### Conditionnelles

-   On additionne les complexités (ou on prend le maximum):

    ```python
    if condition:  # complexité : O(g(n))
        bloc1      # complexité : O(f1(n))
    else:
        bloc2      # complexité : O(f2(n))
    ```

    Coût total en $O(g(n) + f_1(n) + f_2(n)) = O(\max(g(n), f_1(n), f_2(n))$

#### Boucles

-   On **multiplie** la complexité du corps par le
    nombre d’itérations;

-   La complexité d’une boucle `while` se calcule comme suit:

    ```python
    while condition:  # complexité : O(g(n))
        bloc          # complexité : O(f(n))
    ```

    Coût total pour $m$ itérations : $O(m \times \max(g(n), f(n)))$  
    *Attention :* $m$ peut dépendre de $n$ !

#### Appels de fonctions

-   On calcule la complexité du corps de la fonction  
    (exprimée en fonction de la taille de ses paramètres)

-   On substitue cette complexité à la ligne de l'appel

    ```python
    def f(parametres):  # m mesure la taille des parametres
        corps           # complexité : O(f(m))

    mes_params = ...  # n mesure la taille de mes_params
    f(mes_params)     # complexité O(f(n))
    ```

    *Attention :* bien choisir comment on mesure la taille des paramètres !

### Calcul de la complexité d’un algorithme

-   Pour calculer la complexité d’un algorithme:

    1.  on calcule la complexité de chaque “partie” de l’algorithme;

    2.  on combine ces complexités conformément aux règles qu’on vient
        de voir;

    3.  on simplifie le résultat grâce aux règles de simplifications
        qu’on a vues;

        -   élimination des constantes, et

        -   conservation du (des) terme(s) dominant(s)

#### Exemple

Reprenons le calcul de la factorielle, qui nécessitait $5n-1$
opérations:

```python
def fact_iter(n):  
    fact = 1             # affectation :  O(1)
    i = 2                # affectation :  O(1)
    while i <= n:        # comparaison :  O(1)
        fact = fact * i  # mult. + aff. : O(1)
        i = i + 1        # add. + aff. :  O(1)
    return fact          # retour:        O(1)
````

Complexité totale : $O(1 + 1 + (n-1) \times (1 + 1 + 1)) + 1) = O(3n + 3 - 3) = O(n)$

#### Autre méthode possible

On peut aussi calculer le nombre de fois où une des instructions les plus fréquentes est effectuée

- Plus facile à calculer
- Le nombre total aura au plus la même vitesse de croissance (à un facteur près)

Dans `fact_iter`, on peut par exemple compter le nombre de multiplications :

```python
def fact_iter(n):  
    fact = 1
    i = 2        
    while i <= n:
        fact = fact * i  # une multiplication
        i = i + 1
    return fact
````

On obtient bien $n-1$ tours de boucles, donc $n-1$ multiplications, donc $O(n)$ opérations au total.

Justification :
- Les opérations en-dehors de la boucle ne s'exécutent qu'un nombre constant de fois, elles ne contribuent donc pas à l'expression $O(...)$. 
- Chaque tour de boucle effectue un nombre constant d'opérations. Il est donc suffisant d'en compter une choisie arbitrairement.

### Notation $\Theta$

On dit que deux fonctions $f(n)$ et $g(n)$ sont **asymptotiquement équivalentes à un facteur près** et on note $f(n) \in \Theta(g(n))$ si
    
$$f(n) \in O(g(n)) \mbox{ et } g(n) \in O(f(n))$$

autrement dit :

$$\displaystyle
\exists\ n_0 \in \mathbb{N},
\exists\ c_1, c_2 \in \mathbb{R},
  \forall\ n\geq n_0: \quad
  c_1 |g(n)| \leq |f(n)| \leq c_2 |g(n)|
$$

Par exemple, $5n^2 + 30n + 6 \in \Theta(n^2)$.

Cette notation est plus précise que la notation $O$, puisqu'elle affirme qu'une fonction $f$ est à la fois minorée **et** majorée asymptotiquement par une autre fonction $g$. On l'utilise donc à chaque fois que c'est possible.

### Quelques familles de fonctions de complexité

| **Majorant asymptotique**    | **Nom de classe** | **Exemple**
|------------------------------|-------------------|-------------------------
| $O(1)$                       | constante         | minimum d'une liste triée
| $O(\log n)$                  | logarithmique     | recherche dans une liste triée (à suivre)
| $O(n)$                       | linéaire          | recherche dans une liste non triée
| $O(n \log n)$                | quasi-linéaire    | tri optimal d'une liste (à suivre)
| $O(n^2)$                     | quadratique       | recherche de doublons, tri naïf
| $O(n^3)$                     | cubique           | produit de matrices $n \times n$
| $O(2^n)$                     | exponentiel       | tours de Hanoï, sac à dos
  

<img src='img/plot_classes-0.png' style='width:40%'>

<img src='img/plot_classes-1.png' style='width:40%'>

<img src='img/plot_classes-2.png' style='width:40%'>

<img src='img/plot_classes-3.png' style='width:40%'>

<img src='img/plot_classes-4.png' style='width:40%'>

<img src='img/plot_classes-5.png' style='width:40%'>

<img src='img/plot_classes-6.png' style='width:40%'>

## Exemples

### Exemple 1 : somme des $n$ premiers entiers

In [ ]:
def somme_entiers(n):
    acc = 0
    for i in range(n):
        acc = acc + i
    return acc

somme_entiers(10)

**Hypothèses :** `+`, affectations à `i` et à `acc`, `range(n)`, `return` en $O(1)$

**Complexité :** $n$ exécutions du corps de la boucle, donc $n \times O(1) = O(n)$.

In [ ]:
def somme_entiers(n):
    if n == 0:
        return 0
    else:
        return (n-1) + somme_entiers(n-1)

somme_entiers(10)

**Complexité :** 

\begin{align*}
T(n) &= O(1) + T(n-1) \\&= O(1) + \cdots + O(1) \text{\quad($n$ fois)} \\ &= n \times O(1) \\&= O(n)
\end{align*}

### Exemple 2 : liste des couples d'entiers entre $0$ et $n-1$

In [ ]:
def liste_couples(n):
    acc = []
    for i in range(n):
        for j in range(n):
            acc.append((i, j))
    return acc

liste_couples(10)

**Hypothèses :** les précédentes, plus : création d'une liste vide et `append` en $O(1)$, fabrication d'un couple en $O(1)$ 

**Complexité :** pour chaque valeur fixée de `i`, $n$ tours de la petite boucle. Total :

$$
O(\sum_{i=0}^{n-1} n) = O(n \times n) = O(n^2)
$$

### Exemple 3 : recherche de doublons dans une liste

In [ ]:
def est_sans_doublon(lst):
    for i in range(n):
        for j in range(i+1, n):   # /!\
            if list[i] == lst[j]:
                return False
    return True

**Hypothèses :** les précédentes, plus : accès élément de liste en $O(1)$, comparaison en $O(1)$ (**pas toujours vrai !!**) 

**Complexité (au pire !) :** pour chaque valeur fixée de `i`, $n-i-1$ tours de la petite boucle. Total :

\begin{align*}
\sum_{i=0}^{n-1} (n-i-1) &= \sum_{k=n-1}^{0} k & & \text{\quad en posant $k = n-i-1$}\\
                         &= \sum_{k=0}^{n-i-1} k \\
                         &= 0 + 1 + 2 + 3 + \cdots + (n-1)\\
                         &= \frac{n (n-1)}{2}
\end{align*}

Comme chaque tour de la petite boucle est en $O(1)$ on obtient une complexité totale au pire de $O(n^2)$

In [ ]:
# Listes récursives : toutes ces fonctions sont en O(1)

def liste_vide():
    return None

def est_vide(r_liste):
    return r_liste == None

def tete(r_liste):
    return r_liste[0]

def suite(r_liste):
    return r_liste[1]

In [ ]:
def recherche(elem, rlst):
    if est_vide(rlst):
        return False
    elif elem == tete(rlst):
        return True
    else:
        return recherche(elem, suite(rlst))

def contient_doublon(rlst):
    if est_vide(rlst):
        return False
    elif recherche(tete(rlst), suite(rlst)):
        return True
    else:
        return contient_doublon(suite(rlst))

est_sans_doublon((3, (1, (2, (4, (7, None))))))

**Complexité dans le pire cas :**

- `recherche(elem, rlst)` est en $O(k)$ où $k$ est la longueur de `rlst` (Cf. exemple 1)
- `est_sans_doublon` fait un appel à `recherche(elem, rlst)` avec `rlst` de longueur $n-1$, puis $n-2$, etc. 
- Total : $O(n-1 + \cdots + 1 + 0) = O(n^2)$ (même calcul que précédemment)

### Exemple 4 : recherche récursive buggée sur une liste Python

In [ ]:
def recherche_rec(lst, e):
    if len(lst) == 0:
        return False
    elif lst[0] == e:
        return True
    else:
        return recherche_rec(lst[1:], e)

**Hypothèse :** `lst[1:]` compte comme $n-1$ affectations (recopie de tous les éléments sauf un)

**Complexité (au pire !) :** On calcule $n$ fois `lst[1:]` pour `lst` de taille $n$ jusqu'à 1. Total :

\begin{align*}
\sum_{i=n}^{1} (i-1)  &= \sum_{i=n-1}^{0} i \\
                      &= \sum_{i=0}^{n-1} i \\
                      &= 0 + 1 + 2 + \cdots + (n-1)\\
                      &= \frac{n (n-1)}{2}
\end{align*}

Ceci confirme nos observations de début de chapitre ! Cette version est de complexité $O(n^2)$ au lieu de $O(n)$ (d'où la courbe à l'allure de parabole)

### Exemple 5 : test de primalité naïf

In [ ]:
def est_premier(n):
    for i in range(2, n):  # ne pas commencer à 0 ou à 1 !!
        if n % i == 0:
            return False
    return True

est_premier(331999)

**Complexité au pire :** $O(n)$ sous les hypothèses habituelles

**Proposition :** pour tout $n >= 2$, si $n$ n'est pas premier alors il existe $a, b \in \mathbb{N}$, avec $2 \leq a \leq \sqrt{n}$, tels que $n = ab$

In [ ]:
def est_premier_bis(n):
    i = 2
    while i*i <= n:
        if n % i == 0:
            return False
        i += 1
    return True

est_premier_bis(331999)

**Complexité :** ??

## Exercice (tiré du TP 6) : puissance rapide

On souhaite tester et comparer plusieurs algorithmes calculant $a^n$, pour $a$ flottant ou entier et $n$ entier positif. Pour cela, on considère plusieurs définitions de $a^n$.

$$
a^n=\begin{cases}
a*a^{n-1} & \text{ si $n>0$},\\
1 & \text{ si $n=0$}.
\end{cases}
$$


$$
a^n=\begin{cases}
(a^2)^{n/2}         & \text{ si $n>0$ est pair},\\
a*((a^2)^{(n-1)/2}) & \text{ si $n$ est impair},\\
1 & \text{ si $n=0$}.
\end{cases}
$$


$$
a^n=\begin{cases}
(a^{n/2})^2       & \text{ si $n>0$ est pair},\\
a*(a^{(n-1)/2})^2 & \text{ si $n$ est impair},\\
1 & \text{ si $n=0$}.
\end{cases}
$$

Chacune de ces définitions fournit un algorithme récursif. Pour chacun de ces trois algorithmes :

1. Appliquez à la main l'algorithme pour calculer $a^{10}$ pour $a$ quelconque, et comptez le nombre de multiplications nécessaires
2. Cherchez une expression du nombre total de multiplications nécessaires pour calculer $a^n$ en fonction de $n$
3. Donnez une implémentation en Python de l'algorithme